# Dimensionality reduction of Airfoils

### First of all, settings for Jupyter Notebook are used, that affect the inline plots and adjust the width of the Cells

In [1]:
#JupyterNotebook Stuff
%matplotlib inline
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:85% !important; }</style>"))

### Standard imports

In [2]:
#Standard
import numpy as np
import pandas as pd
import os, sys , glob

#Model Persistance
from joblib import dump, load

#Scipy
from scipy import interpolate

#SciKit Learn
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Tensorflow
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
import tensorflow as tf

#Plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.io as pio
import plotly

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)

#Widgets
from ipywidgets import Button, HBox, VBox, Layout, Checkbox, SelectionSlider, IntSlider, FloatSlider, interactive


### General settings for the analysis

In [3]:
reRead_from_TXTs=True # If True, reading in the data from the source textfiles, otherwise using the already imported data from .joblib file


#TO Implement
ProfilesSet='all' # chose 'all', or 'symmetrical' 


### Read all airfoil data
- all the original input data comes from: https://m-selig.ae.illinois.edu/ads/coord_database.html
- the chord length of the airfoils was normalized and the resolution upscaled

In [4]:
#Data Import
if reRead_from_TXTs:
    asymm_airfoils_path='./all_profiles_database_351_points_normalized'
    symm_airfoils_path='./all_profiles_database_351_points_normalized/symmetrical'

    all_input_data_dict={}
    for airfoil_file in glob.glob(asymm_airfoils_path+'/*.dat', recursive=False):
        name=airfoil_file.replace(".dat", "").replace(asymm_airfoils_path+'/','')
        all_input_data_dict[name]={'path':airfoil_file, 'symmetrical':False}

    for airfoil_file in glob.glob(symm_airfoils_path+'/*.dat', recursive=False):
        name=airfoil_file.replace(".dat", "").replace(symm_airfoils_path+'/','')
        all_input_data_dict[name]={'path':airfoil_file, 'symmetrical':True}

    for airfoil_name, airfoil_data in  all_input_data_dict.items():
        coord_DF=pd.read_csv(airfoil_data['path'],skiprows=1,header=None, delim_whitespace= True,)
        coord_DF.rename(columns={0: 'X_coord', 1: 'Y_coord'},inplace=True )
        airfoil_data['coord_DF']=coord_DF
        
    with open('all_input_data_dict.joblib.lzma', 'wb') as f:
        dump(all_input_data_dict, f, compress=('lzma', 9))

else:
    #Read data from .joblib file
    with open('all_input_data_dict.joblib.lzma', "rb") as f:
        all_input_data_dict = load(f)

### Resample to desired X_Coords
For the dimensionality reduction, it is important, that all the Airoils are described in an identical way.
Therefore it is necessary to downsample the resolution and to obtain identical coordinates in the X-direction.
Therefore only the Y-Coordinates can be later used for the dimensionality reduction

X_coord is made monotonic by finding the leading edge Index and interpolated to desired X_coord

In [5]:
targetXcoord=np.loadtxt('X_Coordinates.csv',skiprows=1,delimiter=';',usecols=1)
targetXcoord[101:]=-1*targetXcoord[101:]

def monotonic(x):
    dx = np.diff(x)
    return np.all(dx <= 0) or np.all(dx >= 0)

DimReducion_Input_DF=pd.DataFrame()
DimReducion_Input_DF_index_list=[]
for airfoil_name, airfoil_data in all_input_data_dict.items():
    profile_Xdata=airfoil_data['coord_DF']['X_coord'].values
    profile_Ydata=airfoil_data['coord_DF']['Y_coord'].values
    LeadingEdgeIndex=np.argmin(profile_Xdata)
    profile_Xdata_Mono=airfoil_data['coord_DF']['X_coord'].values*1
    profile_Xdata_Mono[LeadingEdgeIndex:]=-1*profile_Xdata[LeadingEdgeIndex:]
    InterpFunction=interpolate.interp1d(profile_Xdata_Mono,profile_Ydata,fill_value='extrapolate')
    check=InterpFunction(targetXcoord[99])-InterpFunction(targetXcoord[101])
    if (monotonic(profile_Xdata_Mono)==False):
        print(airfoil_name+"\t"+str(monotonic(profile_Xdata_Mono))+"\t"+str(check)+"\t"+str(profile_Ydata[LeadingEdgeIndex]))
    DimReducion_Input_DF=DimReducion_Input_DF.append(pd.DataFrame([InterpFunction(targetXcoord)]))
    DimReducion_Input_DF_index_list.append(airfoil_name)
DimReducion_Input_DF.index=DimReducion_Input_DF_index_list    

### Plot comparison for original and resampled shapes

In [6]:
plotAirfoilName='naca2411-il'

fig = go.Figure(data=[go.Scatter(x=all_input_data_dict[plotAirfoilName]['coord_DF']['X_coord'].values,
                                y=all_input_data_dict[plotAirfoilName]['coord_DF']['Y_coord'].values,
                                mode='lines+markers',
                                name='Orig_351_Coords',),
                      go.Scatter(x=np.loadtxt('X_Coordinates.csv',skiprows=1,delimiter=';',usecols=1),
                                y=DimReducion_Input_DF.loc[plotAirfoilName].values,
                                mode='lines+markers',
                                name='Resampled_201_Coords',),
                     ])
fig.show()

### Calculate informative profile characteristics

In [7]:
AllProfileCharacteristics=pd.DataFrame([],columns=['MaxThickness','MaxThicknessPos','MaxCamber','MaxCamberPos','TE_Thickness','TE_Grad'])
for index, row in DimReducion_Input_DF.iterrows():
    ProfileThickness=np.zeros(len(targetXcoord)//2)
    ProfileCamberLine=np.zeros(len(targetXcoord)//2)
    for i in range (0, len(targetXcoord)//2):
        ProfileThickness[i]=row.values[i]-row.values[-i-1]
        ProfileCamberLine[i]=0.5*(row.values[i]+row.values[-i-1])
    MaxThickness=np.amax(ProfileThickness)
    MaxThicknessPos=targetXcoord[np.argmax(ProfileThickness)]
    TE_Thickness=ProfileThickness[0]
    TE_Grad=(row.values[1]-row.values[0])/(targetXcoord[1]-targetXcoord[0])
    MaxCamber=np.amax(ProfileCamberLine)
    MaxCamberPos=targetXcoord[np.argmax(ProfileCamberLine)]
    AllProfileCharacteristics=AllProfileCharacteristics.append(pd.DataFrame([[MaxThickness,MaxThicknessPos,MaxCamber,MaxCamberPos,TE_Thickness,TE_Grad]],
                                                                            columns=['MaxThickness','MaxThicknessPos','MaxCamber','MaxCamberPos','TE_Thickness','TE_Grad'],
                                                                            index=[index]))

### Split data for testing and validation

In [8]:
x_train, x_test,index_train, index_test = train_test_split(DimReducion_Input_DF.values,
                                                           DimReducion_Input_DF.index,
                                                           test_size=0.1,
                                                           random_state=11,
                                                           shuffle=True)

### PCA Analysis
* PCA is trained with the full number of eigenvectors/eigenvalues
* reduction to same number of latent variables can be done later in comparison to Autoencoder

In [9]:
pca_pipe = make_pipeline(StandardScaler(with_mean=False,with_std=False), PCA(svd_solver='full',whiten=False))
pca_pipe.fit(x_train)
pca = pca_pipe.named_steps['pca']

### Autoencoder
* according to the theory ( https://arxiv.org/pdf/1804.10253.pdf), Autoencoder with linear activation function and one hidden layer makes a projection to the same subspace as PCA with the same number of components.
* however, the obtained latent space variables of autoencoder are not necesarily independent (orthogonal) to each other, like in the case of PCA
* nevertheless, the obtained accuracy of the reduced model should be same as with PCA

In [10]:
latent_dim = 4
ae_scaler = StandardScaler(with_mean=True, with_std=False)

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Dense(latent_dim, activation='linear'),
    ])
    self.decoder = tf.keras.Sequential([
        layers.Dense(201, activation='linear'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

opt = Adam(learning_rate=0.001)
autoencoder.compile(optimizer=opt,
                    loss=losses.MeanSquaredError(),
                    #metrics='accuracy',
                   )

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=1,#len(x_train),
                shuffle=True,
                validation_data=(x_test, x_test),
               verbose=1)

Epoch 1/100
1471/1471 [==============================] - 2s 990us/step - loss: 4.3392e-04 - val_loss: 4.1191e-05
Epoch 2/100
1471/1471 [==============================] - 1s 605us/step - loss: 5.9560e-05 - val_loss: 3.7644e-05
Epoch 3/100
1471/1471 [==============================] - 1s 580us/step - loss: 4.0357e-05 - val_loss: 2.9646e-05
Epoch 4/100
1471/1471 [==============================] - 1s 596us/step - loss: 3.5221e-05 - val_loss: 2.2362e-05
Epoch 5/100
1471/1471 [==============================] - 1s 585us/step - loss: 2.0238e-05 - val_loss: 1.3934e-05
Epoch 6/100
1471/1471 [==============================] - 1s 586us/step - loss: 1.2539e-05 - val_loss: 1.4440e-05
Epoch 7/100
1471/1471 [==============================] - 1s 579us/step - loss: 1.5545e-05 - val_loss: 1.0190e-05
Epoch 8/100
1471/1471 [==============================] - 1s 611us/step - loss: 1.7657e-05 - val_loss: 1.0890e-05
Epoch 9/100
1471/1471 [==============================] - 1s 602us/step - loss: 2.1487e-05 - val_

Epoch 73/100
1471/1471 [==============================] - 1s 589us/step - loss: 1.5241e-05 - val_loss: 1.0831e-05
Epoch 74/100
1471/1471 [==============================] - 1s 581us/step - loss: 1.5029e-05 - val_loss: 1.1145e-05
Epoch 75/100
1471/1471 [==============================] - 1s 581us/step - loss: 1.3558e-05 - val_loss: 1.0614e-05
Epoch 76/100
1471/1471 [==============================] - 1s 593us/step - loss: 1.4405e-05 - val_loss: 1.3037e-05
Epoch 77/100
1471/1471 [==============================] - 1s 580us/step - loss: 1.4928e-05 - val_loss: 1.1139e-05
Epoch 78/100
1471/1471 [==============================] - 1s 610us/step - loss: 1.1996e-05 - val_loss: 1.3105e-05
Epoch 79/100
1471/1471 [==============================] - 1s 614us/step - loss: 1.4214e-05 - val_loss: 1.2069e-05
Epoch 80/100
1471/1471 [==============================] - 1s 616us/step - loss: 1.3242e-05 - val_loss: 1.2323e-05
Epoch 81/100
1471/1471 [==============================] - 1s 609us/step - loss: 1.3631e-

### Comparison PCA vs Autoencoder:

In [11]:
# PCA only with number of components corresponding to latent dimension of AutoEncoder
reduced_PCA=pca_pipe.transform(DimReducion_Input_DF.values)
reduced_PCA[:,latent_dim:]=0
reduced_PCA_DF=pd.DataFrame(reduced_PCA, index=DimReducion_Input_DF.index)
PCA_y_coords_DF=pd.DataFrame(pca_pipe.inverse_transform(reduced_PCA_DF),index=DimReducion_Input_DF.index)

# Autoencoder
encoded_airfiols = pd.DataFrame(autoencoder.encoder(DimReducion_Input_DF.values).numpy(),
                                index=DimReducion_Input_DF.index)
AE_y_coords_DF = pd.DataFrame(autoencoder.decoder(encoded_airfiols.values).numpy(),
                                index=DimReducion_Input_DF.index)

In [12]:
print ("AE Error:             "+str(mean_squared_error(DimReducion_Input_DF.loc[index_test],
                                                        AE_y_coords_DF.loc[index_test])))
      
print ("PCA Error:            "+str(mean_squared_error(DimReducion_Input_DF.loc[index_test],
                                                        PCA_y_coords_DF.loc[index_test])))


AE Error:             1.177836958974834e-05
PCA Error:            9.240483054593824e-06


### Graphical comparison for selected airfoil:

In [32]:
airfoil_to_plot_Name='goe481-il'# 'naca23012-il' '919-Evo_Flap'
                                        # particularly bad fit e.g.: ah93w480b-il_mine , fx79w660a-il, goe481-il

x_coords=np.loadtxt('X_Coordinates.csv',skiprows=1,delimiter=';',usecols=1)

fig = go.Figure(data=[go.Scatter(x=x_coords,
                                 y=DimReducion_Input_DF.loc[airfoil_to_plot_Name],
                                 mode = 'markers',
                                 name='GroundTruth'),
                      go.Scatter(x=x_coords,
                                 y=AE_y_coords_DF.loc[airfoil_to_plot_Name],
                                 mode = 'markers',
                                 name = 'AE'),
                      go.Scatter(x=x_coords,
                                 y=PCA_y_coords_DF.loc[airfoil_to_plot_Name],
                                 mode = 'markers',
                                 name = 'PCA'),
                     ],
                layout = go.Layout(title=go.layout.Title(text="AirfoilName: "+airfoil_to_plot_Name,xref='paper',x=0.5))
               )
iplot(fig)

### Questions:
* why is the accuracy of Autoencoder always lower than PCA?
* how to achieve better accuracy of Autoencoder compared to PCA?
    * which non-linear activation function would be appropriate?
    * would more hidden layers improve the accuracy? (of course not with linear activation function)
    * would any other scaling/preprocessing of the input data be beneficial?
    * could convolution layers bring some benefits?
* any recomendations on Optimizer / Batchsize / Epochs / Learning rate?
* if the variational Autoencoder is used (for the purpose of dis-entanglement), could also similar results to PCA be achevied as the consequence of disentanglement?

In [33]:
pd.DataFrame((AE_y_coords_DF-DimReducion_Input_DF)**2).sum(axis=1).sort_values()

sd6060-il                  0.000065
psu-90-125wl-il            0.000098
s8066-il                   0.000111
s7055-il                   0.000121
s6061-il                   0.000142
                             ...   
goe481-il                  0.034338
griffith30symsuction-il    0.036887
fx79w470a-il_mine          0.078448
fx79w660a-il               0.218926
ah93w480b-il_mine          0.332504
Length: 1635, dtype: float64